#  RAG 체인 구성

---

## RAG란 무엇인가?

### 🎯 핵심 개념
**Retrieval Augmented Generation (RAG)** 는 대규모 언어 모델(LLM)에 외부 지식을 연결하여 더 정확하고 최신의 정보를 제공하는 AI 프레임워크입니다.

### 🔍 RAG의 작동 원리
```
사용자 질문 → 관련 문서 검색 → 컨텍스트와 함께 LLM에 전달 → 답변 생성
```

### 📊 RAG vs 일반 LLM 비교
| 구분 | 일반 LLM | RAG |
|------|----------|-----|
| 정보 소스 | 사전 훈련 데이터만 | 외부 지식베이스 + 사전 훈련 데이터 |
| 최신성 | 훈련 시점까지 | 실시간 업데이트 가능 |
| 정확성 | 환각(hallucination) 가능성 | 검증된 문서 기반 답변 |
| 사용 사례 | 일반적인 질문 답변 | 특정 도메인의 전문적 답변 |

---

## 환경 설정

### 🛠️ 필수 라이브러리 설치

```bash
# 기본 LangChain 패키지
pip install langchain langchain-community 

# 임베딩 모델
pip install langchain-openai langchain-huggingface

# 벡터 저장소
pip install langchain-chroma

# 문서 처리
pip install pypdf 

# 웹 스크래핑
pip install beautifulsoup4

# 토크나이저
pip install tiktoken transformers sentence-transformers

# 실험적 기능 (SemanticChunker)
pip install langchain-experimental

### 🔑 환경 변수 설정

In [ ]:
# .env 파일 생성
from dotenv import load_dotenv
import os

load_dotenv()

# OpenAI API 키 설정 (필요시)
# OPENAI_API_KEY=your_openai_api_key_here

### 📋 기본 라이브러리 import


In [ ]:
import os
from glob import glob
from pprint import pprint
import json
from pathlib import Path

---

## 문서 로더 (Document Loaders)

### 🎯 문서 로더란?
**Document Loader**는 다양한 소스에서 문서를 로드하여 LangChain의 `Document` 객체로 변환하는 도구입니다.

### 📄 Document 객체 구조
```python
from langchain_core.documents import Document

# Document 객체의 기본 구조
document = Document(
    page_content="문서의 텍스트 내용",
    metadata={
        "source": "문서 출처",
        "page": 1,
        "title": "문서 제목"
    }
)
```

### 📄 문서 로더의 종류
- PDF 파일 로더
- 웹 페이지 로더 
- CSV 데이터 로더
- 디렉토리 로더
- HTML 데이터 로더
- JSON 데이터 로더
- Markdown 데이터 로더
- Microsoft Office 데이터 로더


### 1. 🌐 웹 문서 로더 (WebBaseLoader)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

# 기본 웹 문서 로드
web_loader = WebBaseLoader(
    web_paths=[
        "https://python.langchain.com/docs/tutorials/rag/",
        "https://js.langchain.com/docs/tutorials/rag/"
    ]
)

# 문서 로드
web_docs = web_loader.load()
print(f"로드된 문서 수: {len(web_docs)}")

In [ ]:
print(f"첫 번째 문서 내용:\n{web_docs[0].page_content[:500]}...")  # 앞부분만 출력

In [ ]:
print(f"첫 번째 문서 메타데이터: {web_docs[0].metadata}")

### 2. 📊 CSV 파일 로더 (CSVLoader)

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# 기본 CSV 로드
csv_loader = CSVLoader("./data/kbo_teams_2023.csv")
csv_docs = csv_loader.load()

print(f"문서 수: {len(csv_docs)}")

In [ ]:
print(f"첫 번째 문서 내용:\n{csv_docs[0].page_content}")

In [ ]:
print(f"첫 번째 문서 메타데이터: {csv_docs[0].metadata}")

In [ ]:
# 소스 컬럼 지정 및 인코딩 설정
csv_loader_advanced = CSVLoader(
    file_path="./data/kbo_teams_2023.csv",
    source_column="Team",      # 이 컬럼이 메타데이터의 source가 됨
    content_columns=["Team", "Introduction"],  # 이 컬럼이 문서의 내용이 됨
    metadata_columns=["Founded", "City"],  # 이 컬럼이 메타데이터에 추가됨
    encoding="utf-8",          # 인코딩 명시
    csv_args={
        "delimiter": ",",      # 구분자
        "quotechar": '"',      # 인용 문자
    }
)

csv_docs_advanced = csv_loader_advanced.load()

# 문서 수와 첫 번째 문서 내용 출력
print(f"문서 수: {len(csv_docs_advanced)}")

In [ ]:
print(f"마지막 문서 내용:\n{csv_docs_advanced[-1].page_content}")

In [ ]:
print(f"마지막 문서 메타데이터: {csv_docs_advanced[-1].metadata}")

### 3. 📖 PDF 파일 로더 

- **PyPDFLoader**

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/labor_law.pdf')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

In [ ]:
# 각 페이지별 정보 확인
for i, doc in enumerate(pdf_docs[:3]):
    print(f"페이지 {i+1}: {len(doc.page_content)} 문자")
    print(f"메타데이터: {doc.metadata}")

In [ ]:
print(pdf_docs[0].page_content[:1000])  # 첫 페이지의 내용 일부 출력

- **다른 PDF 로더들**

In [ ]:
from langchain_community.document_loaders import (
    PyMuPDFLoader,
    PDFMinerLoader
)

In [ ]:
# PyMuPDF 로더 (빠른 처리)
pymupdf_loader = PyMuPDFLoader("./data/labor_law.pdf")
pdf_docs = pymupdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

# 각 페이지별 정보 확인
for i, doc in enumerate(pdf_docs[:3]):
    print(f"페이지 {i+1}: {len(doc.page_content)} 문자")
    print(f"메타데이터: {doc.metadata}")

In [ ]:
print(pdf_docs[0].page_content[:1000])  # 첫 페이지의 내용 일부 출력

In [ ]:
# PDFMiner 로더 (정확한 텍스트 추출)
pdfminer_loader = PDFMinerLoader("./data/labor_law.pdf")
pdf_docs = pdfminer_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

# 각 페이지별 정보 확인
for i, doc in enumerate(pdf_docs[:3]):
    print(f"페이지 {i+1}: {len(doc.page_content)} 문자")
    print(f"메타데이터: {doc.metadata}")

In [ ]:
print(pdf_docs[0].page_content[:1000])  # 첫 페이지의 내용 일부 출력

### 4. 📝 텍스트 파일 로더 (TextLoader)

In [ ]:
from langchain_community.document_loaders import TextLoader

# 단일 텍스트 파일 로드
text_loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
text_docs = text_loader.load()

print(f"문서 수: {len(text_docs)}")
print(f"첫 번째 문서 내용:\n{text_docs[0].page_content[:1000]}")  # 첫 1000자 출력
print(f"첫 번째 문서 메타데이터: {text_docs[0].metadata}")

In [ ]:
# 디렉토리 내 모든 텍스트 파일 로드
from langchain_community.document_loaders import DirectoryLoader

directory_loader = DirectoryLoader(
    "./data/",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)
all_text_docs = directory_loader.load()

print(f"전체 문서 수: {len(all_text_docs)}")
print(f"첫 번째 문서 내용:\n{all_text_docs[0].page_content[:1000]}")  # 첫 1000자 출력
print(f"첫 번째 문서 메타데이터: {all_text_docs[0].metadata}")

### 🎯 실습 1: 웹 문서 로더 연습

In [ ]:
# 다음 웹 페이지들을 로드하고 메타데이터를 출력해보세요
urls = [
    "https://python.langchain.com/docs/tutorials/",
    "https://python.langchain.com/docs/concepts/"
]

# 여기에 코드를 작성하세요

---

## 텍스트 분할 (Text Splitting)

### 🎯 텍스트 분할이 필요한 이유
1. **토큰 제한**: LLM은 입력 토큰 수에 제한이 있음
2. **검색 정확도**: 작은 청크가 더 정확한 검색 결과 제공
3. **메모리 효율성**: 대용량 문서의 효율적 처리

### 📊 분할 전략 비교
| 방법 | 장점 | 단점 | 사용 사례 |
|------|------|------|----------|
| CharacterTextSplitter | 단순, 빠름 | 문맥 고려 안함 | 간단한 텍스트 |
| RecursiveCharacterTextSplitter | 문맥 보존 우수 | 계산 복잡 | 일반적인 문서 |
| SemanticChunker | 의미 기반 분할 | 느림, 비용 많음 | 중요한 문서 |
| TokenTextSplitter | 정확한 토큰 수 | 토크나이저 의존 | API 비용 최적화 |

In [ ]:
# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/labor_law.pdf', mode='single')  # 'single' 또는 'page' 모드 선택 가능

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

### 1. CharacterTextSplitter

In [ ]:
long_text = pdf_docs[0].page_content
print(f'첫 번째 문서의 텍스트 길이: {len(long_text)}')

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# 기본 설정
text_splitter = CharacterTextSplitter(
    separator="\n\n",        # 구분자
    chunk_size=1000,         # 청크 크기
    chunk_overlap=200,       # 중복 크기
    length_function=len,     # 길이 측정 함수
    is_separator_regex=False # 정규식 여부
)

# 텍스트 분할
chunks = text_splitter.split_text(long_text)
print(f"분할된 청크 수: {len(chunks)}")

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# 기본 설정
text_splitter = CharacterTextSplitter(
    separator="\n",        # 구분자
    chunk_size=1000,         # 청크 크기
    chunk_overlap=200,       # 중복 크기
    length_function=len,     # 길이 측정 함수
    is_separator_regex=False # 정규식 여부
)

# 텍스트 분할
chunks = text_splitter.split_text(long_text)
print(f"분할된 청크 수: {len(chunks)}")

for i, chunk in enumerate(chunks[:10]):
    print(f"청크 {i+1} 길이: {len(chunk)} 문자")

In [ ]:
import re
from langchain.text_splitter import CharacterTextSplitter


# 법률 문서와 공문서에 적합한 문장 분할 정규식 패턴
# 문장 끝 후 공백이 있고, 특정 패턴이 따라오지 않는 경우만 분할 (목록 번호나 조항 번호 앞, 괄호로 된 항목 표시 앞, 기타 구두점 앞)
sentence_pattern = r'(?<=[.!?])\s+(?!\s*(?:\d+|호|조|항|]|\)|[가-힣]{1,2}\s*\)|[A-Za-z]\s*\)|[,;:]))'

# 정규식을 사용한 문장 단위 분할기 생성
sentence_splitter = CharacterTextSplitter(
    separator=sentence_pattern,
    chunk_size=500,
    chunk_overlap=100,
    is_separator_regex=True,
    keep_separator=True
)

# 문장 단위로 분할
sentence_chunks = sentence_splitter.split_text(long_text)
print(f"문장 단위로 분할된 청크 수: {len(sentence_chunks)}")
for i, chunk in enumerate(sentence_chunks[:10]):
    print(f"청크 {i+1} 길이: {len(chunk)} 문자")

In [ ]:
print(f"첫 번째 문서 청크 내용: {sentence_chunks[0]}")

In [ ]:
print(f"두 번째 문서 청크 내용: {sentence_chunks[1]}")

In [ ]:
print(f"세 번째 문서 청크 내용: {sentence_chunks[2]}")

### 2. RecursiveCharacterTextSplitter

- 재귀적으로 텍스트를 분할하여 문맥을 최대한 보존하는 분할 도구


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 기본 재귀 분할기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # 우선순위 순서 (큰 구분자부터 작은 구분자 순서로 재귀적 분할)
)

# Document 객체 분할
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 청크 수: {len(chunks)}")

# 각 청크의 길이 확인
for i, chunk in enumerate(chunks[:3]):
    print(f"청크 {i+1}: {len(chunk.page_content)} 문자")

### 3. 토큰 기반 분할

#### 🔧 TikToken 토크나이저 기반 분할
- OpenAI 임베딩 모델이 사용하는 토크나이저를 사용하여 정확한 토큰 수로 텍스트를 분할하는 도구

In [ ]:
# OpenAI 토크나이저 사용
token_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",  # GPT-4 인코딩
    chunk_size=500,              # 토큰 수 기준
    chunk_overlap=100
)

chunks = token_splitter.split_documents([pdf_docs[0]])

# 토큰 수 확인
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")

for i, chunk in enumerate(chunks[:3]):
    token_count = len(tokenizer.encode(chunk.page_content))
    print(f"청크 {i+1}: {token_count} 토큰, {len(chunk.page_content)} 문자")

#### 🤗 Hugging Face 토크나이저

In [ ]:
from transformers import AutoTokenizer

# BGE-M3 토크나이저 사용
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

hf_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=300,
    chunk_overlap=50
)

chunks = hf_splitter.split_documents([pdf_docs[0]])

# 토큰 수 확인
for i, chunk in enumerate(chunks[:3]):
    token_count = len(tokenizer(chunk.page_content)["input_ids"])
    print(f"청크 {i+1}: {token_count} 토큰, {len(chunk.page_content)} 문자")

### 4. **Semantic Chunking**

- **SemanticChunker**는 텍스트를 의미 단위로 **분할**하는 특수한 텍스트 분할도구 

- 단순 길이 기반이 아닌 **의미 기반**으로 텍스트를 청크화하는데 효과적

- **breakpoint_threshold_type**: Text Splitting의 다양한 임계값(Threshold) 설정 방식 (통계적 기법) 

    - **Gradient** 방식: 임베딩 벡터 간의 **기울기 변화**를 기준으로 텍스트를 분할
    - **Percentile** 방식: 임베딩 거리의 **백분위수**를 기준으로 분할 지점을 결정 (기본값: 95%)
    - **Standard Deviation** 방식: 임베딩 거리의 **표준편차**를 활용하여 유의미한 변화점을 찾아서 분할
    - **Interquartile** 방식: 임베딩 거리의 **사분위수 범위**를 기준으로 이상치를 감지하여 분할

- 설치: pip install langchain_experimental 또는 uv add langchain_experimental


In [ ]:
from langchain_experimental.text_splitter import SemanticChunker 
from langchain_openai.embeddings import OpenAIEmbeddings

# 임베딩 모델을 사용하여 SemanticChunker를 초기화 
text_splitter = SemanticChunker(
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),         # OpenAI 임베딩 사용
    breakpoint_threshold_type="gradient",  # 임계값 타입 설정 (gradient, percentile, standard_deviation, interquartile)
)

In [ ]:
chunks = text_splitter.split_documents(pdf_docs)

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

### 🎯 실습 2: 텍스트 분할 비교

In [ ]:
# 다음 텍스트를 다양한 방법으로 분할하고 결과를 비교해보세요
sample_text = """
인공지능은 현대 기술의 핵심입니다. 
머신러닝을 통해 컴퓨터는 학습할 수 있습니다.

딥러닝은 신경망을 기반으로 합니다.
자연어 처리는 텍스트를 이해하는 기술입니다.

컴퓨터 비전은 이미지를 분석합니다.
강화학습은 행동을 통해 학습합니다.
"""

# 여기에 코드를 작성하세요

---

## 문서 임베딩 (Document Embedding)

### 🎯 임베딩이란?
텍스트를 고차원 벡터 공간의 숫자 배열로 변환하여 의미적 유사도를 계산할 수 있게 하는 기술입니다.

### 📊 임베딩 모델 비교
| 모델 | 차원 | 언어 지원 | 비용 | 성능 | 사용 사례 |
|------|------|----------|------|------|----------|
| OpenAI text-embedding-3-small | 1536 | 다국어 | 유료 | 높음 | 프로덕션 |
| OpenAI text-embedding-3-large | 3072 | 다국어 | 유료 | 최고 | 고성능 요구 |
| BAAI/bge-m3 | 1024 | 다국어 | 무료 | 높음 | 한국어 특화 |
| sentence-transformers/all-MiniLM-L6-v2 | 384 | 영어 | 무료 | 중간 | 로컬 개발 |

### 1. OpenAI 임베딩

#### 🔧 기본 설정

In [ ]:
from langchain_openai import OpenAIEmbeddings

# 기본 임베딩 모델
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1536,           # 차원 수 (기본값)
    show_progress_bar=True,    # 진행률 표시
    max_retries=3             # 재시도 횟수
)

print(f"임베딩 차원: {embeddings_model.dimensions}")
print(f"컨텍스트 길이: {embeddings_model.embedding_ctx_length}")

#### 📝 문서 임베딩

In [ ]:
# 문서 컬렉션 임베딩
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하는 기술입니다.",
    "컴퓨터 비전은 이미지를 분석하는 기술입니다."
]

# 배치 임베딩 (효율적)
doc_embeddings = embeddings_model.embed_documents(documents)
print(f"임베딩 벡터 수: {len(doc_embeddings)}")
print(f"각 벡터 차원: {len(doc_embeddings[0])}")

In [ ]:
# 쿼리 임베딩
query = "AI 기술에 대해 알려주세요"
query_embedding = embeddings_model.embed_query(query)
print(f"쿼리 임베딩 차원: {len(query_embedding)}")

#### 💡 차원 축소 활용

In [ ]:
# 비용 절약을 위한 차원 축소
compact_embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
    dimensions=512  # 원래 1536에서 512로 축소
)

# 성능과 비용의 균형점 찾는 것이 중요!!!
compact_doc_embeddings = compact_embeddings.embed_documents(documents)
print(f"축소된 임베딩 벡터 수: {len(compact_doc_embeddings)}")
print(f"축소된 각 벡터 차원: {len(compact_doc_embeddings[0])}") 

### 2. Hugging Face 임베딩

#### 🤗 BGE-M3 모델 (한국어 우수)


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# BGE-M3 모델 (다국어, 한국어 성능 우수)
embeddings_bge = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cpu'},        # 'cuda' for GPU
    encode_kwargs={'normalize_embeddings': True}  # L2 정규화 - 벡터의 각 차원을 벡터의 L2 노름(크기)으로 나누어 단위 벡터로 변환 (벡터 크기 1로 정규화)
)

# BGE-M3 모델로 문서 임베딩
bge_hf_embeddings = embeddings_bge.embed_documents(documents)
print(f"한국어 임베딩 차원: {len(bge_hf_embeddings[0])}")

#### 📱 경량 모델

In [ ]:
# 빠른 처리를 위한 경량 모델
embedding_gte = HuggingFaceEmbeddings(
    model_name="Alibaba-NLP/gte-multilingual-base",
    model_kwargs={'device': 'cpu', 'trust_remote_code': True},  # trust_remote_code 필요 
    encode_kwargs={'normalize_embeddings': True}
)
    
# 경량 모델로 문서 임베딩
alibaba_hf_embeddings = embedding_gte.embed_documents(documents)
print(f"경량 모델 한국어 임베딩 차원: {len(alibaba_hf_embeddings[0])}") 

### 3. Ollama 임베딩 (로컬)

In [ ]:
from langchain_ollama import OllamaEmbeddings

# Ollama 서버가 실행 중이어야 함
embeddings_ollama = OllamaEmbeddings(
    model="bge-m3",                    # 사용할 모델
    # base_url="http://localhost:11434"  # Ollama 서버 주소
)

# 로컬 임베딩
local_embeddings = embeddings_ollama.embed_documents(documents)

print(f"로컬 임베딩 벡터 수: {len(local_embeddings)}")
print(f"각 벡터 차원: {len(local_embeddings[0])}")

### 4. 유사도 계산 및 검색

#### 📏 코사인 유사도

In [ ]:
from langchain_community.utils.math import cosine_similarity
import numpy as np

def find_most_similar(query, doc_embeddings, documents, embeddings_model):
    """가장 유사한 문서 찾기"""
    # 쿼리 임베딩
    query_embedding = embeddings_model.embed_query(query)
    
    # 코사인 유사도 계산
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    
    # 가장 유사한 문서 인덱스
    most_similar_idx = np.argmax(similarities)
    
    return {
        "document": documents[most_similar_idx],
        "similarity": similarities[most_similar_idx],
        "index": most_similar_idx
    }

# 쿼리와 문서 임베딩을 사용하여 가장 유사한 문서 찾기 (OpenAI)
query = "딥러닝에 대해 알려주세요"
result = find_most_similar(query, doc_embeddings, documents, embeddings_model)

print(f"쿼리: {query}")
print(f"가장 유사한 문서: {result['document']}")
print(f"유사도 점수: {result['similarity']:.4f}")

In [ ]:
# HuggingFaceEmbeddings를 사용한 유사도 검색 (BGE-M3)
result = find_most_similar(query, bge_hf_embeddings, documents, embeddings_bge)
print(f"쿼리: {query}")
print(f"가장 유사한 문서: {result['document']}")
print(f"유사도 점수: {result['similarity']:.4f}")

In [ ]:
# Alibaba-NLP/gte-multilingual-base 모델로 유사도 검색
result = find_most_similar(query, alibaba_hf_embeddings, documents, embedding_gte)
print(f"쿼리: {query}")
print(f"가장 유사한 문서: {result['document']}")
print(f"유사도 점수: {result['similarity']:.4f}")

In [ ]:
# Ollama 모델로 유사도 검색 (bge-m3)
result = find_most_similar(query, local_embeddings, documents, embeddings_ollama)
print(f"쿼리: {query}")
print(f"가장 유사한 문서: {result['document']}")
print(f"유사도 점수: {result['similarity']:.4f}")

### 🎯 실습 3: 임베딩 모델 비교

In [ ]:
# 다음 질문들에 대해 다른 임베딩 모델들의 검색 성능을 비교해보세요
queries = [
    "기계학습이란 무엇인가요?",
    "이미지 인식 기술에 대해 설명해주세요",
    "언어 모델의 작동 원리는?"
]

# 여기에 코드를 작성하세요

---

## 벡터 저장소 (Vector Store)

### 🎯 벡터 저장소란?
임베딩된 벡터를 효율적으로 저장하고 유사도 기반 검색을 수행하는 특수 데이터베이스

### 📊 벡터 저장소 비교
| 종류 | 장점 | 단점 | 사용 사례 |
|------|------|------|----------|
| Chroma | 설치 간단, 로컬 친화적 | 대용량 처리 한계 | 개발, 프로토타입 |
| FAISS | 매우 빠름, 확장성 우수 | 설정 복잡 | 대용량 검색 |
| Pinecone | 완전 관리형, 고성능 | 유료, 클라우드 의존 | 프로덕션 |
| Weaviate | GraphQL 지원, 하이브리드 검색 | 학습 곡선 | 복합 검색 |

### 🚀 Chroma 설치 및 설정
```bash
pip install langchain-chroma
```

#### 📚 기본 사용법

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 준비
pdf_loader = PyPDFLoader('./data/labor_law.pdf', mode='single')
pdf_docs = pdf_loader.load()

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=500,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 청크 수: {len(chunks)}")

# 임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 저장소 생성
chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="labor_law",
    persist_directory="./local_chroma_db",
    collection_metadata={"hnsw:space": "cosine"}  # 유사도 메트릭
)

print(f"저장된 문서 수: {chroma_db._collection.count()}")

#### 💾 벡터 저장소 로드

In [ ]:
# 기존 벡터 저장소 로드
chroma_db = Chroma(
    collection_name="labor_law",
    embedding_function=embeddings,
    persist_directory="./local_chroma_db"
)

print(f"로드된 문서 수: {chroma_db._collection.count()}")

#### 🔍 기본 검색 기능

In [ ]:
# 1. 유사도 검색
query = "탄력 근로에 대해 설명해주세요"
similar_docs = chroma_db.similarity_search(
    query=query,
    k=5,  # 상위 5개 결과
    filter={"source": "./data/labor_law.pdf"}  # 메타데이터 필터
)

print(f"검색 결과 수: {len(similar_docs)}")
for i, doc in enumerate(similar_docs):
    print(f"결과 {i+1}: {doc.page_content[:100]}...")
    print("-" * 40)

In [ ]:
# 2. 점수와 함께 검색 (유사도 점수 포함)
docs_with_scores = chroma_db.similarity_search_with_score(query, k=3)

for doc, score in docs_with_scores:
    print(f"점수: {score:.4f}")
    print(f"내용: {doc.page_content[:100]}...")
    print("-" * 50)

#### 🎛️ 메타데이터 필터링

In [ ]:
# 복합 필터 조건
filter_criteria = {
    "$and": [
        {"source": {"$eq": "./data/labor_law.pdf"}},
        {"page": {"$gte": 10}}  # 10페이지 이상
    ]
}

filtered_results = chroma_db.similarity_search(
    query=query,
    k=5,
    filter=filter_criteria, 
)

print(f"필터링된 검색 결과 수: {len(filtered_results)}")
for i, doc in enumerate(filtered_results):
    print(f"결과 {i+1}: {doc.page_content[:100]}...")
    print("-" * 40)

#### 🔄 문서 업데이트

In [ ]:
# 새 문서 추가
from langchain_core.documents import Document

new_docs = [Document(page_content="새로운 내용", metadata={"source": "new"})]
chroma_db.add_documents(new_docs, ids=["new_doc_1"])

In [ ]:
# 문서 삭제 (ID 기반)
chroma_db.delete(ids=["new_doc_1"])

In [ ]:
# 전체 컬렉션 삭제
# chroma_db.delete_collection()

### 🎯 실습 4: 벡터 저장소 구축

In [ ]:
# 다음 단계로 나만의 벡터 저장소를 구축해보세요:
# 1. 웹에서 문서 로드
# 2. 적절한 크기로 분할
# 3. 임베딩 및 저장
# 4. 검색 테스트

# 여기에 코드를 작성하세요

---

## 검색기 (Retriever)

### 🎯 Retriever란?
벡터 저장소를 기반으로 사용자 질의에 가장 관련성 높은 문서를 검색하는 인터페이스입니다.

### 📊 검색 전략 비교
| 전략 | 설명 | 장점 | 단점 | 사용 사례 |
|------|------|------|------|----------|
| similarity | 단순 유사도 검색 | 빠름, 직관적 | 다양성 부족 | 일반적인 검색 |
| similarity_score_threshold | 임계값 기반 검색 | 품질 보장 | 결과 수 불안정 | 고품질 결과 필요 |
| mmr | 최대 한계 관련성 | 다양성 우수 | 느림 | 포괄적 정보 필요 |

### 1. 기본 유사도 검색

#### 🔍 Top-K 검색

In [ ]:
# 벡터 저장소를 Retriever로 변환
retriever = chroma_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # 상위 5개 결과
)

# 검색 실행
query = "탄력 근로에 대해 설명해주세요"
retrieved_docs = retriever.invoke(query)

print(f"검색된 문서 수: {len(retrieved_docs)}")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}:")
    print(f"내용: {doc.page_content[:200]}...")
    print(f"출처: {doc.metadata.get('source', 'Unknown')}")
    print("-" * 50)

### 2. 임계값 기반 검색

#### 📏 점수 임계값 설정

In [ ]:
# 유사도 점수 임계값 기반 검색
threshold_retriever = chroma_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.3,  # 0.3 이상의 유사도만
        "k": 10                  # 최대 10개까지
    }
)

retrieved_docs = threshold_retriever.invoke(query)

# 실제 유사도 점수 확인
from langchain_community.utils.math import cosine_similarity

for i, doc in enumerate(retrieved_docs):
    # 실제 유사도 계산
    doc_embedding = embeddings.embed_query(doc.page_content)
    query_embedding = embeddings.embed_query(query)
    similarity = cosine_similarity([query_embedding], [doc_embedding])[0][0]
    
    print(f"문서 {i+1} (유사도: {similarity:.4f}):")
    print(f"{doc.page_content[:100]}...")
    print()

### 3. MMR (Maximal Marginal Relevance) 검색

#### 🎯 다양성을 고려한 검색


In [ ]:
# MMR 검색 - 관련성과 다양성의 균형
mmr_retriever = chroma_db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,                # 최종 반환할 문서 수
        "fetch_k": 20,         # 초기 후보 문서 수
        "lambda_mult": 0.5     # 관련성 vs 다양성 (0=최대 다양성, 1=최대 관련성)
    }
)

mmr_docs = mmr_retriever.invoke(query)

print("MMR 검색 결과:")
for i, doc in enumerate(mmr_docs):
    print(f"문서 {i+1}: {doc.page_content[:150]}...")
    print()

#### 🔧 lambda_mult 파라미터 실험


In [ ]:
# 다양한 lambda_mult 값으로 실험
lambda_values = [0.0, 0.25, 0.5, 0.75, 1.0]

for lambda_val in lambda_values:
    retriever = chroma_db.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 3,
            "fetch_k": 10,
            "lambda_mult": lambda_val
        }
    )
    
    docs = retriever.invoke(query)
    print(f"\nLambda {lambda_val} 결과:")
    print("-" * 40)
    for i, doc in enumerate(docs):
        print(f"  {i+1}. {doc.page_content[:100]}...")
    
    print("=" * 40)

#### 🎛️ 메타데이터 필터링

In [ ]:
# 메타데이터 기반 필터링 retriever
filtered_retriever = chroma_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
        "filter": {
            "source": "./data/labor_law.pdf"
        }
    }
)

filtered_results = filtered_retriever.invoke(query)

print("메타데이터 기반 필터링 결과:")
for i, doc in enumerate(filtered_results):
    print(f"문서 {i+1}: {doc.page_content[:150]}...")
    print()

#### 🔄 동적 검색 파라미터

In [ ]:
class DynamicRetriever:
    def __init__(self, vectorstore, embeddings):
        self.vectorstore = vectorstore
        self.embeddings = embeddings
    
    def retrieve(self, query, search_type="auto", k=5):
        """쿼리 특성에 따라 동적으로 검색 전략 선택"""
        
        # 쿼리 복잡도 분석
        query_length = len(query.split())
        
        if query_length <= 3:
            # 짧은 쿼리: 높은 임계값
            search_type = "similarity_score_threshold"
            search_kwargs = {"score_threshold": 0.25, "k": k}
        elif query_length > 10:
            # 긴 쿼리: MMR로 다양성 확보
            search_type = "mmr"
            search_kwargs = {"k": k, "fetch_k": k*3, "lambda_mult": 0.3}
        else:
            # 중간 길이: 기본 유사도 검색
            search_type = "similarity"
            search_kwargs = {"k": k}
        
        retriever = self.vectorstore.as_retriever(
            search_type=search_type,
            search_kwargs=search_kwargs
        )
        
        return retriever.invoke(query)

# 사용 예시
dynamic_retriever = DynamicRetriever(chroma_db, embeddings)

queries = [
    "탄력근로",  # 짧은 쿼리
    "탄력 근로에 대해 설명해주세요",  # 중간 쿼리
    "탄력 근로 제도의 장점과 단점, 그리고 실제 적용 사례를 포함하여 자세히 설명해주세요"  # 긴 쿼리
]

for query in queries:
    print(f"\n쿼리: {query}")
    print(f"길이: {len(query.split())} 단어")
    docs = dynamic_retriever.retrieve(query)
    print(f"검색 결과: {len(docs)}개 문서")
    print("-" * 40)

### 🎯 실습 5: 검색 전략 비교

In [ ]:
# 같은 질문에 대해 다른 검색 전략들의 결과를 비교해보세요
test_query = "근로시간 단축에 대한 규정은 무엇인가요?"

strategies = {
    "similarity": {"k": 5},
    "similarity_score_threshold": {"score_threshold": 0.3, "k": 10},
    "mmr": {"k": 5, "fetch_k": 15, "lambda_mult": 0.5}
}

# 여기에 코드를 작성하세요

---

## RAG 체인 구현

### 🎯 RAG 체인이란?
검색(Retrieval)과 생성(Generation)을 연결하여 외부 지식을 기반으로 답변을 생성하는 파이프라인

### 🔄 RAG 워크플로우
`
사용자 질문 → 관련 문서 검색 → 컨텍스트 구성 → LLM 프롬프트 → 답변 생성
`

### 1. 프롬프트 템플릿 설계

#### 📝 기본 RAG 프롬프트

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 기본 RAG 프롬프트 템플릿
basic_template = """주어진 컨텍스트를 기반으로 질문에 답변하세요.

컨텍스트:
{context}

질문: {question}

답변:"""

basic_prompt = ChatPromptTemplate.from_template(basic_template)

#### 🎨 고급 RAG 프롬프트

In [ ]:
advanced_template = """당신은 전문적인 문서 분석 AI입니다. 주어진 컨텍스트를 바탕으로 정확하고 유용한 답변을 제공하세요.

## 답변 지침
- 컨텍스트에 있는 정보만을 사용하여 답변하세요
- 확실하지 않은 정보는 "명확하지 않습니다"라고 명시하세요
- 답변은 논리적이고 구조화된 형태로 제공하세요
- 가능한 경우 구체적인 예시나 수치를 포함하세요
- 답변에 참조한 문서의 출처가 있다면 포함하세요 (문서명, 페이지, URL 등)

## 컨텍스트
{context}

## 질문
{question}

## 답변 형식
**핵심 답변:** (질문에 대한 직접적인 답변)

**세부 설명:** (추가적인 설명이나 배경 정보)

**관련 정보:** (컨텍스트에서 발견된 연관 정보)

**답변:**"""

advanced_prompt = ChatPromptTemplate.from_template(advanced_template)

#### 🌟 도메인별 특화 프롬프트

In [ ]:
legal_template = """당신은 법률 문서 전문 AI입니다. 법률 조항을 정확히 해석하고 설명해주세요.

## 법률 해석 원칙
- 조문의 정확한 인용을 포함하세요
- 법적 용어는 일반인이 이해할 수 있도록 설명하세요
- 예외 조항이나 단서가 있다면 반드시 언급하세요
- 관련 법령이나 시행령도 함께 언급하세요

## 관련 법률 조항
{context}

## 법률 질의
{question}

## 법률 답변
**해당 조항:** (관련 법률 조항 인용)

**조항 해석:** (조항의 의미와 적용 범위)

**주의사항:** (예외 조항이나 제한 사항)

**답변:**"""

legal_prompt = ChatPromptTemplate.from_template(legal_template)

### 2. LLM 설정

In [ ]:
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.1,                   # 일관성 있는 답변
    max_completion_tokens=1000,        # 답변 길이 제한
    top_p=0.9,              # 다양성 제어
    frequency_penalty=0.1,  # 반복 방지
    presence_penalty=0.1    # 새로운 단어 장려
)

### 3. RAG 체인 구성

#### 🔗 기본 LCEL 체인

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    """문서 리스트를 문자열로 포맷"""
    return "\n\n".join([
        f"{doc.page_content}" for doc in docs
    ])

# 기본 RAG 체인
basic_rag_chain = (
    RunnableParallel({
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    })
    | basic_prompt
    | llm
    | StrOutputParser()
)

# 테스트
query = "탄력 근로에 대해 설명해주세요"
result = basic_rag_chain.invoke(query)
print(result)

#### 🎯 고급 RAG 체인

In [ ]:
def format_docs_with_metadata(docs):
    """메타데이터를 포함한 문서 포맷팅"""
    formatted_docs = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'Unknown')
        page = doc.metadata.get('page', 'N/A')
        
        formatted_doc = f"""
출처: {source}
페이지: {page}
내용: {doc.page_content}
---
"""
        formatted_docs.append(formatted_doc)
    
    return "\n".join(formatted_docs)


# 고급 RAG 체인
advanced_rag_chain = (
    RunnableParallel({
        "context": retriever | format_docs_with_metadata,
        "question": RunnablePassthrough()
    })
    | advanced_prompt
    | llm
    | StrOutputParser()
)

# 테스트
query = "탄력 근로에 대해 설명해주세요"
result = advanced_rag_chain.invoke(query)
print(result)

### 🎯 실습 6: 완전한 RAG 시스템 구축

In [ ]:
# 다음 요구사항을 만족하는 RAG 시스템을 구축해보세요:
# 1. 여러 문서 형식 지원 (PDF, 웹, 텍스트)
# 2. 동적 검색 전략 선택
# 3. 도메인별 프롬프트 템플릿
# 4. 응답 품질 평가

class ComprehensiveRAGSystem:
    def __init__(self):
        pass
    
    def load_documents(self, sources):
        """다양한 소스에서 문서 로드"""
        # 여기에 코드를 작성하세요
        pass
    
    def setup_vector_store(self, documents):
        """벡터 저장소 구성"""
        # 여기에 코드를 작성하세요
        pass
    
    def query(self, question, domain="general"):
        """도메인별 질의응답"""
        # 여기에 코드를 작성하세요
        pass

# 여기에 구현 코드를 작성하세요